In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import importlib as imp

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from itertools import accumulate
from math import floor, ceil, sqrt, log, pi
from matplotlib import pyplot as plt
from tensorflow.keras import layers, utils, losses, models as mds, optimizers

if imp.util.find_spec('aggdraw'): import aggdraw
if imp.util.find_spec('tensorflow_addons'): from tensorflow_addons import layers as tfa_layers
if imp.util.find_spec('tensorflow_models'): from official.vision.beta.ops import augment as visaugment
if imp.util.find_spec('tensorflow_probability'): from tensorflow_probability import distributions as tfd

2022-03-28 14:59:02.181001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 14:59:02.181083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
def mean_per_class_acc(y_true, y_pred):
    score, up_opt = tf.compat.v1.metrics.mean_per_class_accuracy(y_true, y_pred, num_classes=2)
    tf.print(score.eval())

    score = tf.identity(score)       
    return score

# metric = tf.compat.v1.metrics.mean_per_class_accuracy([1, 0], [[.2, .8], [.1, .9]], 2)
with tf.compat.v1.Session() as sess:
    y_true = tf.one_hot([1, 0], 2)
    y_pred = tf.constant([[.2, .8], [.1, .9]], shape=(2,2))
    score = mean_per_class_acc(y_true, y_pred)
    print(score)

# Signed IoU Metric

* It produces a positive value for overlapping and a negative value for non-overlapping boxes.


##  Sparse Variant

In [ ]:
IOU_EPSILON = 0.0000000000001/(IMG_SIZE*IMG_SIZE)

def compute_signed_iou(boxes_1, boxes_2):
    """
        It computes the IoU of boxes in boxes_1 with the corresponding boxes in boxes_2.
        A negative IoU is a measure of the non-overlap between the corresponding boxes.

        Arguments:
            boxes_1: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
            boxes_2: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            * Intersections of boxes in boxes_1 and boxes_2 in YXHW format. Shape: (N_BOXES, 4)
            * IoUs of shape (N_BOXES, 1)
    """
    [b1_yx_min, b1_yx_max] = tf.split(boxes_1, 2, axis=-1)
    [b2_yx_min, b2_yx_max] = tf.split(boxes_2, 2, axis=-1)

    # Pick maximum yx_min and minimum yx_max to get the intersection
    yx_min = tf.where(b1_yx_min > b2_yx_min, b1_yx_min, b2_yx_min)
    yx_max = tf.where(b1_yx_max < b2_yx_max, b1_yx_max, b2_yx_max)

    intersection_yxyx = tf.concat([yx_min, yx_max], axis=-1)
    intersection_yxhw = yxyx_to_yxhw(intersection_yxyx)

    # Compute intersection height and width
    y_min, x_min = tf.split(yx_min, 2, axis=-1)
    y_max, x_max = tf.split(yx_max, 2, axis=-1)
    intersection_h, intersection_w = y_max - y_min, x_max - x_min

    # Record negative heights and widths. We'll force a negative area
    # if both of them are negative.
    negative_h_intersection = intersection_h < 0
    negative_w_intersection = intersection_w < 0
    negative_hw_intersection = tf.math.logical_and(negative_h_intersection, negative_w_intersection)

    # Compute intersection and union and use them to compute IoUs
    intersection = tf.where(negative_hw_intersection, intersection_h*-intersection_w, intersection_h*intersection_w)
    b1_area, b2_area = box_area(boxes_1), box_area(boxes_2)
    union = b1_area + b2_area - intersection
    iou = intersection/(union + IOU_EPSILON)

    return intersection_yxhw, iou

def evaluate_boxes(boxes, predictions):
    """
        It computes the IoU of boxes and predictions using compute_signed_iou() function

        Arguments:
            boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
            predictions: A tensor of boxes with shape (IMG_SIZE, IMG_SIZE, 2) in HW format.

        Returns:
            * A tensor of predictions in YXHW format. Shape (N_BOXES, 4)
            * Intersections of boxes with predictions in YXHW format. Shape: (N_BOXES, 4)
            * IoUs of shape (N_BOXES, 1)
    """
    boxes = boxes.to_tensor()
    size = predictions.shape[0]

    boxes_yx_min = boxes[:, :2]
    yx_min_indices = yx_to_indices(boxes_yx_min, size)

    # tf.print('boxes_yx_min: ', boxes_yx_min, boxes_yx_min.shape)
    # tf.print('yx_min_indices: ', yx_min_indices, yx_min_indices.shape)

    hw_preds = tf.gather_nd(predictions, yx_min_indices)[:, :2]
    yxhw_preds = tf.concat([boxes_yx_min, hw_preds], axis=-1)
    yxyx_preds = yxhw_to_yxyx(yxhw_preds)

    # tf.print('hw_preds: ', hw_preds, hw_preds.shape)
    # tf.print('yxhw_preds: ', yxhw_preds, yxhw_preds.shape)

    yxhw_intersection, iou = compute_signed_iou(boxes, yxyx_preds)

    # return yxhw_preds, yxhw_intersection, tf.reduce_mean(iou)
    return yxhw_preds, yxhw_intersection, iou

def evaluate_predictions(y_true, y_pred):
    """
        It computes mean IoUs for a batch.

        Arguments:
            y_true: A ragged tensor of true boxes in YXYX format. Shape (BATCH_SIZE, N_BOXES*, 4)
            y_pred: A tensor of predicted boxes in HW format. Shape (BATCH_SIZE, IMG_SIZE, IMG_SIZE, 2)

        Returns:
            * A ragged tensor of predictions in YXHW format. Shape (BATCH_SIZE, N_BOXES*, 4)
            * Intersections of boxes with predictions in YXHW format. Shape: (BATCH_SIZE, N_BOXES*, 4)
            * Mean IoU of shape (1)

    """
    batch_size = tf.shape(y_true)[0]

    yxhw_intersections = []
    yxhw_preds = []
    ious = []

    for item_id in range(batch_size):
    # item_id = 0
        yxhw_pred, yxhw_intersection, iou = evaluate_boxes(y_true[item_id], y_pred[item_id])
        
        yxhw_intersections.append(yxhw_intersection)
        yxhw_preds.append(yxhw_pred)
        ious.append(iou)
    
    mean_iou = tf.reduce_mean(tf.concat(ious, axis=0))

    # tf.print('mean_iou: ', mean_iou)
    
    return yxhw_preds, yxhw_intersections, mean_iou

## Dense Variant

### YXHW Format

In [ ]:
def box_iou(boxes_1, boxes_2):
    """
        It computes the IoU of boxes in boxes_1 with the corresponding boxes in boxes_2.
        A negative IoU is a measure of the non-overlap between the corresponding boxes.
        boxes_1: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
        boxes_2: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tuple of:
            * Intersections of boxes in boxes_1 and boxes_2 in YXHW format. Shape: (N_BOXES, 4)
            * IoUs of shape (N_BOXES, 1)
    """
    [b1_yx_min, b1_yx_max] = tf.split(boxes_1, 2, axis=-1)
    [b2_yx_min, b2_yx_max] = tf.split(boxes_2, 2, axis=-1)

    # Pick maximum yx_min and minimum yx_max to get the intersection
    yx_min = tf.where(b1_yx_min > b2_yx_min, b1_yx_min, b2_yx_min)
    yx_max = tf.where(b1_yx_max < b2_yx_max, b1_yx_max, b2_yx_max)

    intersection_yxyx = tf.concat([yx_min, yx_max], axis=-1)
    intersection_yxhw = yxyx_to_yxhw(intersection_yxyx)

    # Compute intersection height and width
    y_min, x_min = tf.split(yx_min, 2, axis=-1)
    y_max, x_max = tf.split(yx_max, 2, axis=-1)
    intersection_h, intersection_w = y_max - y_min, x_max - x_min

    # Record negative heights and widths. We'll force a negative area
    # if both of them are negative.
    negative_h_intersection = intersection_h < 0
    negative_w_intersection = intersection_w < 0
    negative_hw_intersection = tf.math.logical_and(negative_h_intersection, negative_w_intersection)

    # Compute intersection and union and use them to compute IoUs
    intersection = tf.where(negative_hw_intersection, intersection_h*-intersection_w, intersection_h*intersection_w)
    b1_area, b2_area = box_area(boxes_1), box_area(boxes_2)
    union = b1_area + b2_area - intersection
    iou = tf.math.divide_no_nan(intersection, union)

    return intersection_yxhw, iou

def compute_iou(y_true, y_pred):
    batch_size = y_true.shape[0]
    intersections, ious, box_counts = [], [], []

    # fig, axes = plt.subplots(1, 2, figsize=(8, 3))
    # axes = axes.ravel()

    for item_id in range(batch_size):
        # item_id = 0

        yxhw_true = hw_grid_to_yxhw(y_true[item_id])
        
        # Extract relevant boxes from predictions based on the true box count
        num_boxes = tf.shape(yxhw_true)[0]
        num_non_boxes = MAX_BOXES - num_boxes
        yxhw_pred = y_pred[item_id][:num_boxes]

        # tf.print('yxhw_true: ', yxhw_true, yxhw_true.shape)
        # tf.print('yxhw_pred: ', yxhw_pred, yxhw_pred.shape)

        # Transform YXHWs to YXYXs
        yxyx_true = yxhw_to_yxyx(yxhw_true)
        yxyx_pred = yxhw_to_yxyx(yxhw_pred)

        # Compute IoUs
        intersection_yxhw, iou = box_iou(yxyx_true, yxyx_pred)

        # tf.print('yxyx_true: ', yxyx_true, yxyx_true.shape)
        # tf.print('yxyx_pred: ', yxyx_pred, yxyx_pred.shape)

        # We align the intersections to MAX_BOXES.
        # This ensure we always return identically shaped tensors
        non_box_intersection_yxhw = tf.zeros((num_non_boxes, intersection_yxhw.shape[-1]))
        intersection_yxhw_combined = tf.concat([intersection_yxhw, non_box_intersection_yxhw], axis=0)
        intersections.append(intersection_yxhw_combined)

        # tf.print('intersection_yxhw_combined: ', intersection_yxhw_combined, intersection_yxhw_combined.shape)

        # We align the IoUs to MAX_BOXES.
        # This ensure we always return identically shaped tensors
        non_box_iou = tf.zeros((num_non_boxes, 1))
        combined_iou = tf.concat([iou, non_box_iou], axis=0)
        ious.append(combined_iou)

        # tf.print('combined_iou: ', combined_iou, combined_iou.shape)

        # We record box counts to inform the box count to the caller.
        box_counts.append(num_boxes)

        # axes[item_id].set_xlim([0, IMG_SIZE])
        # axes[item_id].set_ylim([IMG_SIZE, 0])

        # plot_boxes(axes[item_id], yxhw_true, IMG_SIZE, color='red')
        # plot_boxes(axes[item_id], yxhw_pred, IMG_SIZE)
        # plot_boxes(axes[item_id], intersection_yxhw, IMG_SIZE, fill=True, color='blue', alpha=0.5)

    return tf.stack(intersections, axis=0), tf.stack(ious, axis=0), tf.stack(box_counts, axis=0)

def compute_iou_metric(y_true, y_pred):
    _, ious, box_counts = compute_iou(y_true, y_pred)
    batch_size = tf.shape(ious)[0]
    positive_ious, positive_iou_counts = 0., 0.
    negative_ious, negative_iou_counts = 0., 0.

    for item_id in range(batch_size):
        # item_id = 0
        # Slice IoUs for boxes and discard the non-boxes which are zeroes.
        item_box_count = box_counts[item_id]
        item_iou = ious[item_id, :item_box_count]

        # tf.print('item_iou: ', item_iou, item_iou.shape)

        # Create box masks for positive and negative IoU values.
        # These will be used to average them separately.
        item_negative_iou_mask = tf.cast(tf.math.less(item_iou, 0), dtype=tf.float32)
        item_positive_iou_mask = 1. - item_negative_iou_mask

        # tf.print('item_positive_iou_mask: ', item_positive_iou_mask, item_positive_iou_mask.shape)
        # tf.print('item_negative_iou_mask: ', item_negative_iou_mask, item_negative_iou_mask.shape)

        # Count the number of positive and negative IoU items. These will be used
        # for reduction.
        item_positive_iou_count = tf.math.reduce_sum(item_positive_iou_mask)
        item_negative_iou_count = tf.math.reduce_sum(item_negative_iou_mask)

        # tf.print('item_positive_iou_count: ', item_positive_iou_count, item_positive_iou_count.shape)
        # tf.print('item_negative_iou_count: ', item_negative_iou_count, item_negative_iou_count.shape)

        # Average positive and negative IoUs individually.
        item_positive_iou = tf.math.divide_no_nan(item_iou*item_positive_iou_mask, item_positive_iou_count)
        item_negative_iou = tf.math.divide_no_nan(item_iou*item_negative_iou_mask, item_negative_iou_count)

        # tf.print('item_positive_iou: ', item_positive_iou, item_positive_iou.shape)
        # tf.print('item_negative_iou: ', item_negative_iou, item_negative_iou.shape)

        positive_ious += tf.reduce_sum(item_positive_iou)
        negative_ious += tf.reduce_sum(item_negative_iou)

        positive_iou_counts += item_positive_iou_count
        negative_iou_counts += item_negative_iou_count

    mean_positive_iou = tf.math.divide_no_nan(positive_ious, positive_iou_counts)
    mean_negative_iou = tf.math.divide_no_nan(negative_ious, negative_iou_counts)
    mean_iou = (positive_ious + negative_ious)/(positive_iou_counts + negative_iou_counts)

    # tf.print('mean_positive_iou: ', mean_positive_iou, mean_positive_iou.shape)
    # tf.print('mean_negative_iou: ', mean_negative_iou, mean_negative_iou.shape)

    return mean_iou, mean_positive_iou, mean_negative_iou

# itr = iter(train_prep_ds.batch(2))
# images, y_true = next(itr)
# y_pred = tf.random.uniform((y_true.shape[0], MAX_BOXES, 4))

# compute_iou_metric(y_true, y_pred)

### CYCXHW Format

In [ ]:
def box_iou(boxes_1, boxes_2):
    """
        It computes the IoU of boxes in boxes_1 with the corresponding boxes in boxes_2.
        A negative IoU is a measure of the non-overlap between the corresponding boxes.
        boxes_1: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
        boxes_2: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tuple of:
            * Intersections of boxes in boxes_1 and boxes_2 in YXHW format. Shape: (N_BOXES, 4)
            * IoUs of shape (N_BOXES, 1)
    """
    [b1_yx_min, b1_yx_max] = tf.split(boxes_1, 2, axis=-1)
    [b2_yx_min, b2_yx_max] = tf.split(boxes_2, 2, axis=-1)

    # Pick maximum yx_min and minimum yx_max to get the intersection
    yx_min = tf.where(b1_yx_min > b2_yx_min, b1_yx_min, b2_yx_min)
    yx_max = tf.where(b1_yx_max < b2_yx_max, b1_yx_max, b2_yx_max)

    intersection_yxyx = tf.concat([yx_min, yx_max], axis=-1)
    intersection_yxhw = yxyx_to_yxhw(intersection_yxyx)

    # Compute intersection height and width
    y_min, x_min = tf.split(yx_min, 2, axis=-1)
    y_max, x_max = tf.split(yx_max, 2, axis=-1)
    intersection_h, intersection_w = y_max - y_min, x_max - x_min

    # Record negative heights and widths. We'll force a negative area
    # if both of them are negative.
    negative_h_intersection = intersection_h < 0
    negative_w_intersection = intersection_w < 0
    negative_hw_intersection = tf.math.logical_and(negative_h_intersection, negative_w_intersection)

    # Compute intersection and union and use them to compute
    # IoUs
    intersection = tf.where(negative_hw_intersection, intersection_h*-intersection_w, intersection_h*intersection_w)
    b1_area, b2_area = box_area(boxes_1), box_area(boxes_2)
    union = b1_area + b2_area - intersection
    iou = tf.math.divide_no_nan(intersection, union)

    return intersection_yxhw, iou

def compute_iou(y_true, y_pred):
    """
        It computes IoUs for a batch of boxes using box_iou functions.
    """
    batch_size = y_true.shape[0]
    intersections, ious, box_counts = [], [], []

    # fig, axes = plt.subplots(1, 2, figsize=(8, 3))
    # axes = axes.ravel()

    for item_id in range(batch_size):
        # item_id = 0
        cycxhw_true = hw_grid_to_cycxhw(y_true[item_id])
        
        # Extract relevant boxes from predictions based on the true box count
        num_boxes = tf.shape(cycxhw_true)[0]
        num_non_boxes = MAX_BOXES - num_boxes
        cycxhw_pred = y_pred[item_id][:num_boxes]

        # tf.print('cycxhw_true: ', cycxhw_true, cycxhw_true.shape)
        # tf.print('cycxhw_pred: ', cycxhw_pred, cycxhw_pred.shape)

        # Transform CYCXHWs to YXYXs
        yxyx_true = cycxhw_to_yxyx(cycxhw_true)
        yxyx_pred = cycxhw_to_yxyx(cycxhw_pred)

        # Compute IoUs
        intersection_yxhw, iou = box_iou(yxyx_true, yxyx_pred)
        intersection_cycxhw = yxhw_to_cycxhw(intersection_yxhw)

        # tf.print('yxyx_true: ', yxyx_true, yxyx_true.shape)
        # tf.print('yxyx_pred: ', yxyx_pred, yxyx_pred.shape)

        # We align the intersections to MAX_BOXES.
        # This ensure we always return identically shaped tensors
        non_box_intersection_cycxhw = tf.zeros((num_non_boxes, intersection_cycxhw.shape[-1]))
        intersection_cycxhw_combined = tf.concat([intersection_cycxhw, non_box_intersection_cycxhw], axis=0)
        intersections.append(intersection_cycxhw_combined)

        # tf.print('intersection_cycxhw_combined: ', intersection_cycxhw_combined, intersection_cycxhw_combined.shape)

        # We align the IoUs to MAX_BOXES.
        # This ensure we always return identically shaped tensors
        non_box_iou = tf.zeros((num_non_boxes, 1))
        combined_iou = tf.concat([iou, non_box_iou], axis=0)
        ious.append(combined_iou)

        # tf.print('combined_iou: ', combined_iou, combined_iou.shape)

        # We record box counts to inform the box count to the caller.
        box_counts.append(num_boxes)

        # axes[item_id].set_xlim([0, IMG_SIZE])
        # axes[item_id].set_ylim([IMG_SIZE, 0])

        # plot_box_centers(axes[item_id], cycxhw_true, IMG_SIZE, color='red')
        # plot_box_centers(axes[item_id], cycxhw_pred, IMG_SIZE)
        # plot_box_centers(axes[item_id], intersection_cycxhw, IMG_SIZE, color='blue', alpha=0.5)

    return tf.stack(intersections, axis=0), tf.stack(ious, axis=0), tf.stack(box_counts, axis=0)

def compute_iou_metric(y_true, y_pred):
    _, ious, box_counts = compute_iou(y_true, y_pred)
    batch_size = tf.shape(ious)[0]
    positive_ious, positive_iou_counts = 0., 0.
    negative_ious, negative_iou_counts = 0., 0.

    for item_id in range(batch_size):
        # item_id = 0
        # Slice IoUs for boxes and discard the non-boxes which are zeroes.
        item_box_count = box_counts[item_id]
        item_iou = ious[item_id, :item_box_count]

        # tf.print('item_iou: ', item_iou, item_iou.shape)

        # Create box masks for positive and negative IoU values.
        # These will be used to average them separately.
        item_negative_iou_mask = tf.cast(tf.math.less(item_iou, 0), dtype=tf.float32)
        item_positive_iou_mask = 1. - item_negative_iou_mask

        # tf.print('item_positive_iou_mask: ', item_positive_iou_mask, item_positive_iou_mask.shape)
        # tf.print('item_negative_iou_mask: ', item_negative_iou_mask, item_negative_iou_mask.shape)

        # Count the number of positive and negative IoU items. These will be used
        # for reduction.
        item_positive_iou_count = tf.math.reduce_sum(item_positive_iou_mask)
        item_negative_iou_count = tf.math.reduce_sum(item_negative_iou_mask)

        # tf.print('item_positive_iou_count: ', item_positive_iou_count, item_positive_iou_count.shape)
        # tf.print('item_negative_iou_count: ', item_negative_iou_count, item_negative_iou_count.shape)

        # Average positive and negative IoUs individually.
        item_positive_iou = tf.math.divide_no_nan(item_iou*item_positive_iou_mask, item_positive_iou_count)
        item_negative_iou = tf.math.divide_no_nan(item_iou*item_negative_iou_mask, item_negative_iou_count)

        # tf.print('item_positive_iou: ', item_positive_iou, item_positive_iou.shape)
        # tf.print('item_negative_iou: ', item_negative_iou, item_negative_iou.shape)

        positive_ious += tf.reduce_sum(item_positive_iou)
        negative_ious += tf.reduce_sum(item_negative_iou)

        positive_iou_counts += item_positive_iou_count
        negative_iou_counts += item_negative_iou_count

    mean_positive_iou = tf.math.divide_no_nan(positive_ious, positive_iou_counts)
    mean_negative_iou = tf.math.divide_no_nan(negative_ious, negative_iou_counts)
    mean_iou = (positive_ious + negative_ious)/(positive_iou_counts + negative_iou_counts)

    # tf.print('mean_positive_iou: ', mean_positive_iou, mean_positive_iou.shape)
    # tf.print('mean_negative_iou: ', mean_negative_iou, mean_negative_iou.shape)

    return mean_iou, mean_positive_iou, mean_negative_iou

# itr = iter(train_prep_ds.batch(2))
# images, y_true = next(itr)
# y_pred = tf.random.uniform((y_true.shape[0], MAX_BOXES, 4))

# compute_iou_metric(y_true, y_pred)

# Mean Metrics

In [ ]:
class Mean(metrics.Metric):
    def __init__(self, name="mean", **kwargs):
        super(Mean, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total_{}'.format(name), initializer="zeros")
        self.count = self.add_weight(name='count_{}'.format(name), initializer="zeros")

    def update_state(self, result):
        self.total.assign_add(result)
        self.count.assign_add(1)

    def result(self):
        return self.total/self.count

    def reset_state(self):
        # The state of the metric will be reset at the start of each epoch.
        self.total.assign(0.0)
        self.count.assign(0)

# History

In [ ]:
class History(object):
    def __init__(self, names=[]):
        _names = ['learning_rate'] + names + list(map(lambda name: f'val_{name}', names))
        _defs = list(map(lambda name: Mean(name=name), _names))

        self.metrics = dict(zip(_names, _defs))        
        self.history = {name: [] for name, metric in self.metrics.items()}
    
    @property
    def metric_names(self):
        return list(self.metrics.keys())

    @property
    def training_metrics_names(self):
        return list(filter(lambda name: not name.startswith('val_'), self.metrics.keys()))
    
    @property
    def training_metrics(self):
        return [(name, self.metrics[name].result()) for name in self.training_metrics_names]

    @property
    def metric_values(self):
        return [(name, metric.result()) for name, metric in self.metrics.items()]
    
    def train_step(self, metrics):
        for name, value in metrics.items():
            self.metrics[name].update_state(value)

        return self.training_metrics
    
    def val_step(self, metrics):
        for name, value in metrics.items():
            self.metrics[name].update_state(value)

        return self.metric_values
    
    def learning_rate(self, lr_value):
        self.metrics['learning_rate'].update_state(lr_value)
    
    def epoch(self):
        # Record the current epoch values before reset.
        values = self.metric_values

        for name in self.metrics.keys():
            self.record_and_reset(name)
        
        return values
    
    def record_and_reset(self, name):
        self.history[name].append(self.metrics[name].result().numpy())
        self.metrics[name].reset_state()

hist = History(['hello'])
hist.train_step(dict(hello=2.0))
hist.val_step(dict(val_hello=4.0))
hist.epoch()

print(f'Training History: {hist.history}')